## Final Project Submission


* Eric Denbin & Allison Gao
* August 27th, 2021
* House Price Prediction for King County, WA Using Multiple Regression 


In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import gauss
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
import statsmodels
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

%matplotlib inline

## Cleaning/Transforming and Exploring the Data

In [18]:
df = pd.read_csv("data/kc_house_data.csv")
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [19]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19221.000000,21534.000000,21597.000000,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007596,0.233863,3.409825,7.657915,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086825,0.765686,0.650546,1.173200,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [20]:
#created a new column that converted year built into house age 
df["house_age"] = 2021 - df["yr_built"]
df = df.drop(columns=["id","yr_built"])

In [21]:
# detected outliers and dropped them
# eliminated question marks from sqft_basement
df = df[df.bedrooms != 33]
df = df[df.sqft_basement != "?"]

In [22]:
# converted sqft_basement into a binary
# 1 if there is a basement
# 0 if there is no basement

df["sqft_basement"] = df["sqft_basement"].astype(str).astype(float).astype(int)
df["sqft_basement"] = [1 if x > 0 else 0 for x in df["sqft_basement"]]

In [23]:
# converted lat into a binary

#north of the county is 1 and south is zero 
def label_lat (row):
    if row['lat'] >= 47.5000 :
        return 1
    else:
        return 0 


In [24]:
# converted longinto a binary

#west side of the county is 1 and east side is zero 

def label_long (row):
    if row['long'] <= (-122.0000) :
        return 1
    else:
        return 0 


In [25]:
#created a new column for the lat as a dummy

df['county_lat'] = df.apply (lambda row: label_lat(row), axis=1)

In [26]:
#created a new column for the long as a dummy
df['county_long'] = df.apply (lambda row: label_long(row), axis=1)

In [27]:
#logged following variables to normalize the variables with non-normal distributions

non_normal = ['price', 'bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above','sqft_lot15', 'sqft_living15', 'house_age']
for feat in non_normal:
    df[feat] = df[feat].map(lambda x: np.log(x))

In [28]:
# Creating dummies out of season feature

seasons = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]

month_to_season = dict(zip(range(1,13), seasons))

def season(month):
    
    return month_to_season[month]

In [29]:
from datetime import datetime
df['month'] = df['date'].map(lambda x: datetime.strptime(x, "%m/%d/%Y").month)
df['season'] = df['month'].map(lambda x: int(season(x)))

In [30]:
# Creating dummies out of season feature
from sklearn.preprocessing import OneHotEncoder

# Create the OneHotEncoder object
ohe = OneHotEncoder(drop='first')

# Transform the data into dummies
trans = ohe.fit_transform(df[['season']])

# Store the dummies matrix and name vector
data = trans.todense()
names = ohe.get_feature_names()

# Concatenate the dummies df to the original dataset
dummies = pd.DataFrame(data, columns=names)

print(dummies)

df = df.join(dummies)
df.head()

       x0_2  x0_3  x0_4
0       0.0   0.0   1.0
1       0.0   0.0   1.0
2       0.0   0.0   0.0
3       0.0   0.0   1.0
4       0.0   0.0   0.0
...     ...   ...   ...
21137   1.0   0.0   0.0
21138   0.0   0.0   0.0
21139   1.0   0.0   0.0
21140   0.0   0.0   0.0
21141   0.0   0.0   1.0

[21142 rows x 3 columns]


,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_living15,sqft_lot15,house_age,county_lat,county_long,month,season,x0_2,x0_3,x0_4
0,10/13/2014,12.309982,1.098612,1.00,7.073270,8.639411,1.0,NaN,0.0,3,...,7.200425,8.639411,4.189655,1,1,10,4,0.0,0.0,1.0
1,12/9/2014,13.195614,1.098612,2.25,7.851661,8.887653,2.0,0.0,0.0,3,...,7.432484,8.941022,4.248495,1,1,12,4,0.0,0.0,1.0
2,2/25/2015,12.100712,0.693147,1.00,6.646391,9.210340,1.0,0.0,0.0,3,...,7.908387,8.994917,4.477337,1,1,2,1,0.0,0.0,0.0
3,12/9/2014,13.311329,1.386294,3.00,7.580700,8.517193,1.0,0.0,0.0,5,...,7.215240,8.517193,4.025352,1,1,12,4,0.0,0.0,1.0
4,2/18/2015,13.142166,1.098612,2.00,7.426549,8.997147,1.0,0.0,0.0,3,...,7.495542,8.923058,3.526361,1,1,2,1,0.0,0.0,0.0


In [31]:
df.dropna(inplace=True)
df = df.replace([np.inf, -np.inf], np.nan)
df.round(3)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_living15,sqft_lot15,house_age,county_lat,county_long,month,season,x0_2,x0_3,x0_4
1,12/9/2014,13.196,1.099,2.25,7.852,8.888,2.0,0.0,0.0,3,...,7.432,8.941,4.248,1,1,12,4,0.0,0.0,1.0
3,12/9/2014,13.311,1.386,3.00,7.581,8.517,1.0,0.0,0.0,5,...,7.215,8.517,4.025,1,1,12,4,0.0,0.0,1.0
4,2/18/2015,13.142,1.099,2.00,7.427,8.997,1.0,0.0,0.0,3,...,7.496,8.923,3.526,1,1,2,1,0.0,0.0,0.0
5,5/12/2014,14.023,1.386,4.50,8.598,11.532,1.0,0.0,0.0,3,...,8.468,11.532,2.996,1,1,5,2,1.0,0.0,0.0
8,4/15/2015,12.344,1.099,1.00,7.484,8.919,1.0,0.0,0.0,3,...,7.484,9.001,4.111,1,1,4,2,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21134,6/18/2014,14.152,1.386,2.75,8.261,9.215,2.0,0.0,0.0,3,...,8.178,9.215,2.773,1,1,6,2,0.0,0.0,1.0
21135,7/2/2014,12.488,0.693,1.50,6.685,7.659,2.0,0.0,0.0,3,...,6.928,8.476,2.565,1,1,7,3,0.0,0.0,0.0
21136,3/6/2015,13.017,1.099,2.25,7.390,6.963,3.0,0.0,0.0,3,...,7.237,6.848,2.485,1,1,3,1,0.0,0.0,0.0
21137,5/5/2015,13.727,1.386,2.50,7.976,8.379,3.0,0.0,0.0,3,...,7.952,8.338,2.398,1,1,5,2,1.0,0.0,0.0


In [32]:
df['x0_2'] = df['x0_2'].map(lambda x:int(x))
df['x0_3'] = df['x0_3'].map(lambda x:int(x))
df['x0_4'] = df['x0_4'].map(lambda x:int(x))